<a href="https://colab.research.google.com/github/Imnotsure-YT/ConnectFourAI/blob/main/TestConnect4AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Connect 4 Reinforcement Learning Model

##Setup

In [5]:
# Import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from scipy.signal import convolve2d
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Environment
RED = 1
YELLOW = -1
EMPTY = 0

class C4Env:
    def __init__(self, rows=6, cols=7): # 6 rows, 7 cols; stored from 5-0: 0 is top row
        self.rows = rows
        self.cols = cols
        self.shape = (rows, cols)
        self.board = np.zeros(self.shape, dtype=int)
        self.current_player = RED # player 1 and player -1

    def render(self, newline=True):
        display = '\n'.join([''.join(['R' if item==RED else 'Y' if item==YELLOW else '.' for item in row]) for row in self.board])
        if newline: display += '\n'
        print(display)

    def reset(self):
        self.board = np.zeros(self.shape, dtype=int)
        self.current_player = RED
        return self.board

    def legal_actions(self):
        return np.nonzero(self.board[0] == 0)[0] # which columns have empty top

    def step(self, action):
        assert action in self.legal_actions(), "Invalid action"
        # Find the first empty spot in the column
        for row in range(self.rows-1, -1, -1):
            if self.board[row, action] == EMPTY:
                self.board[row, action] = self.current_player
                break
        # Check for a win
        if self.winning_move(self.current_player):
            return self.board, self.current_player, True
        # Switch player
        self.current_player *= -1
        return self.board, None, False

    def winning_move(self, piece):
        # Define the kernel to use for convolution
        kernel = np.array([[1, 1, 1, 1]])

        # Check horizontal
        if (convolve2d(self.board == piece, kernel, mode='valid') == 4).any():
            return True

        # Check vertical
        if (convolve2d(self.board == piece, kernel.T, mode='valid') == 4).any():
            return True

        # Check positively sloped diagonals
        if (convolve2d(self.board == piece, np.eye(4), mode='valid') == 4).any():
            return True

        # Check negatively sloped diagonals
        if (convolve2d(self.board == piece, np.fliplr(np.eye(4)), mode='valid') == 4).any():
            return True

        return False



In [7]:
# Model Hyperparameters

num_episodes = 10000
epsilon = 0.5
ep_decay = 0.9
gamma = 0.9
learning_rate = 0.1

# save path and other misc variables
save_path = "/content/drive/MyDrive/COSMOS 2024/Connect Four/"

# Model parameters

class QNet(nn.Module):
    def __init__(self, rows=6, cols=7):
        super(QNet, self).__init__()
        self.a = 64
        self.b = 64

        self.rows = rows
        self.cols = cols

        self.fc1 = nn.Linear(self.rows * self.cols, self.a)
        self.fc2 = nn.Linear(self.a, self.b)
        self.fc3 = nn.Linear(self.b, self.cols)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Agent

class QLearningAgent:
    def __init__(self, epsilon, ep_decay, gamma, learning_rate):
        self.p_network = QNet()
        self.t_network = QNet()
        self.optimizer = optim.Adam(self.p_network.parameters(), lr=learning_rate)
        self.epsilon = epsilon
        self.ep_decay = ep_decay
        self.gamma = gamma
        self.step = 0

    def get_action(self, state, valid_actions, boardEnv=None):
        if valid_actions.size == 0:
            return -1
        if random.random() > self.epsilon:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state.flatten()).unsqueeze(0)
                q_values = self.p_network(state_tensor)
                valid_q_values = q_values[0][valid_actions]
                return valid_actions[torch.argmax(valid_q_values).item()]
        else:
            return random.choice(valid_actions)


    def update(self, state, action, reward, next_state, done):
        state_tensor = torch.FloatTensor(state.flatten()).unsqueeze(0)
        next_state_tensor = torch.FloatTensor(next_state.flatten()).unsqueeze(0)
        action_tensor = torch.LongTensor([action])
        reward_tensor = torch.FloatTensor([reward])

        q_values = self.p_network(state_tensor)
        next_q_values = self.t_network(next_state_tensor)

        q_value = q_values.gather(1, action_tensor.unsqueeze(1))
        next_q_value = next_q_values.max(1)[0].unsqueeze(1)
        expected_q_value = reward_tensor + (1 - done) * self.gamma * next_q_value

        loss = nn.MSELoss()(q_value, expected_q_value.detach())

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_tnet(self):
        self.t_network.load_state_dict(self.p_network.state_dict())

    def save(self, path):
        torch.save({
            'model_state_dict': self.p_network.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'epsilon': self.epsilon,
            'gamma': self.gamma
        }, path)

    def load(self, path):
        if os.path.exists(path):
            checkpoint = torch.load(path)
            self.p_network.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.epsilon = checkpoint['epsilon']
            self.gamma = checkpoint['gamma']
        else:
            print(f"No saved model found at {path}")

    def epsilon_decay(self):
        self.epsilon = max(self.ep_min, self.epsilon * self.ep_decay)


##Train

In [8]:
# Training loop
def train(num_episodes, epsilon=0.5, ep_decay=0.9, gamma=0.01, learning_rate=0.1):
    agent1 = QLearningAgent(epsilon=epsilon, ep_decay=ep_decay, gamma=gamma, learning_rate=learning_rate)
    agent2 = QLearningAgent(epsilon=epsilon, ep_decay=ep_decay, gamma=gamma, learning_rate=learning_rate)

    for episode in range(num_episodes):
        boardEnv = C4Env()
        board = boardEnv.board
        game_over = False
        turn = 0
        total_turns = 0

        while not game_over:
            # Get Action
            valid_actions = boardEnv.legal_actions()

            if turn == 0:
                action = agent1.get_action(board, valid_actions, boardEnv)
                piece = RED
            else:
                action = agent2.get_action(board, valid_actions, boardEnv)
                piece = YELLOW

            # Take Action
            if action == -1:
                continue
            boardEnv.step(action)

            # Get Rewards
            reward = 0
            if boardEnv.winning_move(piece):
                reward = 1 if turn == 0 else -1
                game_over = True
            elif np.all(boardEnv.legal_actions == False):
                game_over = True

            next_state = board.copy()

            if turn == 0:
                agent1.update(board, action, reward, next_state, game_over)
            else:
                agent2.update(board, action, -reward, next_state, game_over)

            board = next_state
            turn = 1 - turn
            total_turns += 1

        if episode % 1000 == 0:
            print(f"Episode {episode} completed: turns - {total_turns}")
            boardEnv.render()


    return agent1, agent2


Episode 0 completed: turns - 31
Y.Y..YR
R.R..RY
R.R..YY
YRY..RY
RRRYRYR
YRYRRYY

Episode 1000 completed: turns - 9
.......
......R
......R
......R
......R
YYR..YY

Episode 2000 completed: turns - 16
.R.....
.R.....
.R...Y.
.Y...Y.
.R..YYY
RRR.YYR

Episode 3000 completed: turns - 16
.......
.......
R.Y..Y.
Y.Y..R.
RRYR.Y.
RYYR.R.

Episode 4000 completed: turns - 15
....R..
....Y..
..R.R..
..RYY..
..RYY..
RYRYR..

Episode 5000 completed: turns - 11
.......
.......
.R.....
.R....Y
.RRR..Y
.RYY..Y

Episode 6000 completed: turns - 11
.......
.......
..Y.R..
..Y.R..
..Y.R.R
..RYR.Y

Episode 7000 completed: turns - 17
....Y..
....R..
R...Y..
R..YR..
RR.YY.Y
RR.YR.Y

Episode 8000 completed: turns - 13
..R....
..R....
..R....
Y.R....
Y.YR...
YYRY..R

Episode 9000 completed: turns - 11
.......
.......
..R....
..RY...
..RY..Y
R.RY.YR



In [ ]:
# manually train a new model

agent1, agent2 = train(num_episodes)

agent1.load(os.path.join(save_path, "agent1.pth"))
agent2.load(os.path.join(save_path, "agent2.pth"))


##Evaluation

##Player v Computer

In [11]:
# Function to play a game using the trained model
def play_game(turn=0): # turn = 0 -> player goes first; turn = 1 -> agent goes first
    boardEnv = C4Env()
    board = boardEnv.board
    game_over = False
    PLAYER = RED if turn==0 else YELLOW
    AGENT = RED if turn==-1 else YELLOW
    agent = agent2 if turn==0 else agent1

    while not game_over:
        valid_actions = boardEnv.legal_actions()

        if turn == 0:
            action = int(input("Enter Move: "))
            piece = PLAYER
        else:
            action = agent.get_action(board, valid_actions)
            piece = AGENT

        boardEnv.step(action)

        if boardEnv.winning_move(piece):
            boardEnv.render()
            print(f"{'Agent' if piece==AGENT else 'Player'} wins!")
            game_over = True
        elif np.all(boardEnv.legal_actions == False):
            boardEnv.render()
            print("It's a draw!")
            game_over = True

        boardEnv.render()
        turn = 1 - turn

# Play a game with the trained agents
play_game(turn=0)

Enter Move: 3
.......
.......
.......
.......
.......
...R...

.......
.......
.......
.......
.......
.Y.R...

Enter Move: 3
.......
.......
.......
.......
...R...
.Y.R...

.......
.......
.......
.......
...R...
.Y.R.Y.

Enter Move: 3
.......
.......
.......
...R...
...R...
.Y.R.Y.

.......
.......
...Y...
...R...
...R...
.Y.R.Y.

Enter Move: 1
.......
.......
...Y...
...R...
.R.R...
.Y.R.Y.

.......
.......
...Y...
.Y.R...
.R.R...
.Y.R.Y.

Enter Move: 5
.......
.......
...Y...
.Y.R...
.R.R.R.
.Y.R.Y.

.......
...Y...
...Y...
.Y.R...
.R.R.R.
.Y.R.Y.

Enter Move: 2
.......
...Y...
...Y...
.Y.R...
.R.R.R.
.YRR.Y.

.......
...Y...
.Y.Y...
.Y.R...
.R.R.R.
.YRR.Y.

Enter Move: 1
.......
.R.Y...
.Y.Y...
.Y.R...
.R.R.R.
.YRR.Y.

.......
.R.Y...
.Y.Y...
.Y.R.Y.
.R.R.R.
.YRR.Y.

Enter Move: 4
.......
.R.Y...
.Y.Y...
.Y.R.Y.
.R.R.R.
.YRRRY.

.Y.....
.R.Y...
.Y.Y...
.Y.R.Y.
.R.R.R.
.YRRRY.

Enter Move: 4
.Y.....
.R.Y...
.Y.Y...
.Y.R.Y.
.R.RRR.
.YRRRY.

.Y.Y...
.R.Y...
.Y.Y...
.Y.R.Y.
.R.RRR.
.

In [12]:
from google.colab import drive

drive.mount('/content/drive')

agent1.save(os.path.join(save_path, "agent1.pth"))
agent2.save(os.path.join(save_path, "agent2.pth"))

Mounted at /content/drive
